In [2]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import IPython.display as display
import soundfile as sf
import truecase
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import truecase
nlp = spacy.load("en_core_web_sm")

In [3]:
# Loading the audio file
audio, rate = librosa.load("data/short1.wav", sr = 16000)

# printing audio 
print(audio)

# printing rate
print(rate)

[0. 0. 0. ... 0. 0. 0.]
16000


In [4]:
# Importing Wav2Vec pretrained model

tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/Users/christopherdoan/miniconda3/envs/speech/lib/python3.8/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:356: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def asr_transcript(tokenizer, model, input_file):
  transcript = ""
  pre, rate = librosa.load(input_file, sr= 16000)
  sf.write("data/temp1.wav", pre, rate )
  print(librosa.get_samplerate("data/temp1.wav"))

  stream = librosa.stream("data/temp1.wav",
                          block_length = 25,
                          frame_length = 16000,
                          hop_length = 16000)
  
  for speech in stream:
    if len(speech.shape) > 1:
      speech = speech[:,0] + speech[:,1]

    input_values = tokenizer(speech, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim = -1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    print(truecase.get_true_case(transcription.lower()))
    #print(transcription)
    transcript += truecase.get_true_case(transcription.lower()) + "\n"

  return transcript


short1_trans = asr_transcript(tokenizer, model, "data/short1.wav")

16000
At home one where is Jane she is in the living room what is she doing she is playing the piano where is the car it is in the garage where is the dog the dog is in front of the door what is the dog doing the dog is eating at home two
Where are you I am in the kitchen what are you doing I am cooking dinner where are bill and Mary they are in the living room what are they doing they are watching Te V where is the cat she is in the dining room what is she doing she is sleeping my favorite photograph
One who is she she is my sister What's her name her name is Genifer where is she in this photograph She's in Toronto what is that building behind her she's standing in front of the s nd tower location one where is the school It's between the Li
Brarie and the park where is the post office It's across from the movie theatre where is the Royal Bank It's next to the supermarket where is the gas station It's around the corner from the church where is the Barber shop It's near the bus station 

In [6]:
file1 = open("data/speech.txt", "w")
file1.write(short1_trans)
file = open("data/speech.txt", mode="r")
speech = file.read()
doc = nlp(short1_trans) # put the Speech.txt string into the nlp object (this pipeline automatically extracts NER entities)
for entities in doc.ents:
  if (entities.label_ == "CARDINAL"):
    nothing = "nothing"
  else:
    print(f"word: {entities.text}: TAG: {entities.label_}")

word: Jane: TAG: PERSON
word: Mary: TAG: PERSON
word: Te V: TAG: PERSON
word: Genifer: TAG: PERSON
word: Toronto: TAG: GPE
word: Li: TAG: PERSON
word: Brarie: TAG: PERSON
word: the Royal Bank: TAG: ORG
word: Barber: TAG: PERSON
word: Bank: TAG: ORG
word: Geneva Strait: TAG: LOC
word: this year: TAG: DATE
word: Brown: TAG: PERSON
word: English: TAG: LANGUAGE
word: Alice young: TAG: PERSON
word: Albert: TAG: PERSON
word: Julie: TAG: PERSON
word: Jack: TAG: PERSON
word: Dave: TAG: PERSON
word: last night: TAG: TIME
word: about six o'clock: TAG: TIME
word: Loane: TAG: PERSON
word: Friday: TAG: DATE
word: Friday: TAG: DATE
word: evening: TAG: TIME
word: Friday: TAG: DATE
word: Friday: TAG: DATE
word: eleven thirty: TAG: TIME
word: Coathanger: TAG: LOC
word: Hanger: TAG: PERSON
word: Maple Street: TAG: FAC
word: David: TAG: PERSON
word: one week: TAG: DATE
word: Torono: TAG: PERSON
word: six o'clock: TAG: TIME
word: eight thirty: TAG: TIME
word: Reese: TAG: PERSON
word: Huthis: TAG: ORG
word